In [ ]:
import duckdb
import boto3
import pandas as pd
import datetime

s3 = boto3.client('s3', aws_access_key_id='', aws_secret_access_key='')
s3._request_signer.sign = (lambda *args, **kwargs: None)


response = s3.list_objects_v2(
    Bucket='cycling.data.tfl.gov.uk',
    Prefix='usage-stats',

)


In [ ]:
df = pd.DataFrame(response["Contents"])

In [ ]:
files_dict = response["Contents"]
files=[]
for file in files_dict:
    if file["Key"].endswith("csv") and file["LastModified"].year==2025:
        file_key = file["Key"]
        files.append(f"s3://cycling.data.tfl.gov.uk/{file_key}")



In [ ]:
duckdb.read_csv(files, normalize_names=True).to_parquet("/workspaces/de_zoomcamp_2025_project/data/2025_journeys.parquet", append=True)

In [ ]:
[ x for x in files_dict if x["Key"]=="usage-stats/402JourneyDataExtract15Aug2024-26Aug2024.csv"][0]["LastModified"].year
                                                                                                 

In [ ]:
duckdb.read_csv(f"s3://cycling.data.tfl.gov.uk/usage-stats/402JourneyDataExtract15Aug2024-26Aug2024.csv")

In [ ]:
duckdb.sql("select * from read_parquet('data/2025_journeys.parquet')")

In [ ]:

duckdb.read_csv(f"s3://cycling.data.tfl.gov.uk/usage-stats/{file_key}").to_parquet("/workspaces/de_zoomcamp_2025_project/data/2025_journeys.parquet")).to_parquet("/workspaces/de_zoomcamp_2025_project/data/2025_journeys.parquet")